# PYTORCH CNN Classifier

In [1]:
DATASET = 'none'
VOCAB_SIZE = 0
TOKENIZER = 'none'
KMER = 0

In [2]:
# Parameters
DATASET = "human_nontata_promoters"
TOKENIZER = "character"


In [3]:
print(DATASET, VOCAB_SIZE, TOKENIZER, KMER)

human_nontata_promoters 0 character 0


## Config

In [4]:
import torch
from torch.utils.data import DataLoader

from genomic_benchmarks.dataset_getters.pytorch_datasets import get_dataset
from glp.models import CNN
from glp.tokenizers import get_tokenizer
from glp.tokenizers.utils import build_vocab, coll_factory, check_config, check_seq_lengths

In [5]:
config = {
    "dataset": DATASET,
    "tokenizer": TOKENIZER,
    "dataset_version": 0,
    "epochs": 5,
    "batch_size": 32,
    "use_padding": True,
    "force_download": False,
    "run_on_gpu": True,
    "number_of_classes": 2,
    "embedding_dim": 100,
}
check_config(config)

## Choose the dataset

In [6]:
train_dset = get_dataset(config["dataset"], 'train')

## Tokenizer and vocab

In [7]:
tokenizer = get_tokenizer(config['tokenizer'])
tokenizer.train(train_dset=train_dset, vocab_size=VOCAB_SIZE, kmer=KMER)
vocabulary = build_vocab(train_dset, tokenizer, use_padding=config["use_padding"])

print("vocab len:" ,vocabulary.__len__())
print(vocabulary.get_stoi())

vocab len: 8
{'<eos>': 6, 'G': 4, 'A': 3, 'C': 2, '<pad>': 7, 'T': 5, '<bos>': 1, '<unk>': 0}


## Dataloader and batch preparation

In [8]:
# Run on GPU or CPU
device = 'cuda' if config["run_on_gpu"] and torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

max_tok_len = check_seq_lengths(dataset=train_dset, tokenizer=tokenizer)

# Data Loader
collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = max_tok_len)

train_loader = DataLoader(train_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

Using cpu device


max_tok_len  253


In [9]:
tokenizer(train_dset[1][0])

['<bos>',
 'C',
 'A',
 'C',
 'T',
 'C',
 'T',
 'G',
 'T',
 'G',
 'T',
 'T',
 'C',
 'T',
 'T',
 'T',
 'T',
 'A',
 'A',
 'A',
 'A',
 'T',
 'C',
 'C',
 'C',
 'T',
 'C',
 'A',
 'G',
 'G',
 'T',
 'T',
 'T',
 'T',
 'A',
 'T',
 'G',
 'T',
 'T',
 'T',
 'T',
 'A',
 'T',
 'T',
 'G',
 'C',
 'T',
 'A',
 'C',
 'C',
 'A',
 'G',
 'A',
 'G',
 'T',
 'C',
 'T',
 'G',
 'C',
 'C',
 'T',
 'C',
 'C',
 'C',
 'T',
 'G',
 'A',
 'G',
 'G',
 'T',
 'T',
 'C',
 'T',
 'T',
 'G',
 'T',
 'A',
 'T',
 'A',
 'G',
 'A',
 'C',
 'T',
 'A',
 'G',
 'T',
 'T',
 'A',
 'T',
 'T',
 'T',
 'C',
 'C',
 'C',
 'T',
 'C',
 'T',
 'G',
 'T',
 'A',
 'A',
 'A',
 'G',
 'A',
 'A',
 'G',
 'C',
 'T',
 'G',
 'T',
 'T',
 'C',
 'T',
 'A',
 'T',
 'T',
 'C',
 'G',
 'T',
 'T',
 'C',
 'T',
 'C',
 'G',
 'C',
 'C',
 'T',
 'G',
 'G',
 'T',
 'T',
 'T',
 'G',
 'G',
 'A',
 'A',
 'C',
 'A',
 'A',
 'A',
 'C',
 'T',
 'G',
 'A',
 'A',
 'C',
 'A',
 'C',
 'T',
 'T',
 'C',
 'C',
 'A',
 'A',
 'A',
 'G',
 'G',
 'A',
 'G',
 'G',
 'C',
 'A',
 'G',
 'T',
 'C',
 'C',


## Model

In [10]:
model = CNN(
    number_of_classes=config["number_of_classes"],
    vocab_size=vocabulary.__len__(),
    embedding_dim=config["embedding_dim"],
    input_len=max_tok_len
).to(device)

## Training

In [11]:
model.train(train_loader, epochs=config["epochs"])

Epoch 0


/Users/katarina/Documents/repositories/GitHub/GLP/src/glp/tokenizers/utils.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(pad(x), dtype=torch.long)


Train metrics: 
 Accuracy: 81.6%, Avg loss: 0.619230 

Epoch 1


Train metrics: 
 Accuracy: 83.2%, Avg loss: 0.603101 

Epoch 2


Train metrics: 
 Accuracy: 83.5%, Avg loss: 0.603677 

Epoch 3


Train metrics: 
 Accuracy: 84.1%, Avg loss: 0.598727 

Epoch 4


Train metrics: 
 Accuracy: 82.4%, Avg loss: 0.599614 



## Testing

In [12]:
test_dset = get_dataset(config["dataset"], 'test')
test_loader = DataLoader(test_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

acc, f1 = model.test(test_loader)
acc, f1

p  4119 ; tp  2920.390381336212 ; fp  556.8699052144054
recall  0.7090047053498937 ; precision  0.8398538333847849
num_batches 283
correct 7284
size 9034
Test metrics: 
 Accuracy: 0.806287, F1 score: 0.768902, Avg loss: 0.606912 



(0.8062873588665043, 0.7689021363596088)